**The demo does not function under code in this repository!** 

This demo allows you to run summarization of the books. It invokes the LLMs through Brainbox from [Kaia](http://github.com/okulovsky/kaia) project, and thus must be run under properly configured Kaia invironment (see [README.md](http://github.com/okulovsky/kaia/blob/main/README.md)) and with running and configured Brainbox (see [demos](https://github.com/okulovsky/kaia/tree/main/demos/brainbox)). In general, this demo is here only for the reference causes; its outputs are stored permanently in the repo and then pulled to the repository in Demo 2. No thorough documentation will be provided for this part, but it is available in Kaia repository, in [Brainbox demos](https://github.com/okulovsky/kaia/tree/main/demos/brainbox).

In [102]:
from kaia.infra import FileIO

BOOK = 'ru_crime_and_punishment_fragments.json'
input_data = FileIO.read_json(f'./files/fragments/{BOOK}')

In [103]:
from kaia.brainbox import BrainBoxTask, BrainBoxTaskPack, BrainBox
from uuid import uuid4

def fragment_to_task(fragment, prompt_template):
    prompt = prompt_template.format(fragment['text'])
    task = BrainBoxTask(
        id = BrainBoxTask.safe_id(),
        decider = 'Oobabooga',
        decider_parameters= 'openchat_3.5.Q5_K_M.gguf',
        decider_method='chat_completions',
        arguments = dict(prompt=prompt)
    )
    return task

tasks = []
tags = {}
ids = {}

for fragment in input_data['fragments']:
    task = fragment_to_task(fragment, input_data['prompt'])
    tags[task.id] = fragment
    ids[task.id] = task.id
    tasks.append(task)

collection_task = BrainBoxTask(
    id = BrainBoxTask.safe_id(),
    decider = 'Collector',
    decider_method='collect_outputs',
    arguments = dict(tags=tags),
    dependencies = ids
)

pack = BrainBoxTaskPack(collection_task, tasks)
    
api = BrainBox().create_api('127.0.0.1')
api.add(pack)

In [112]:
result = FileIO.read_pickle(api.download(api.get_result('id_d5e06bd9781f436094c7b5b3ce4f94ff')))
assert 0 == len(result['errors'])
result['records']

[{'job_id': 'id_f52b6985afc248f38d75921dcbc14aed',
  'tags': {'id': '6300d8eb-ef53-11ee-ac0a-2c33583ea070',
   'file_id': 'ru_book_0',
   'text_type': 'narrative',
   'text': 'В начале июля, в чрезвычайно жаркое время, под вечер, один молодой человек вышел из своей каморки, которую нанимал от жильцов в С—м переулке, на улицу и медленно, как бы в нерешимости, отправился к К—ну мосту.1\nОн благополучно избегнул встречи с своею хозяйкой на лестнице. Каморка его приходилась под самою кровлей высокого пятиэтажного дома и походила более на шкаф, чем на квартиру. Квартирная же хозяйка его, у которой он нанимал эту каморку с обедом и прислугой, помещалась одною лестницей ниже, в отдельной квартире, и каждый раз, при выходе на улицу, ему непременно надо было проходить мимо хозяйкиной кухни, почти всегда настежь отворенной на лестницу. И каждый раз молодой человек, проходя мимо, чувствовал какое-то болезненное и трусливое ощущение, которого стыдился и от которого морщился. Он был должен кругом х

In [113]:
FileIO.write_pickle(result, f'source/ml_retell/{BOOK}.result.pkl')